In [ ]:
!pip install openai -q
!pip install azure-storage-blob
!pip install langchain
!pip install tiktoken
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 16.7 MB/s eta 0:00:00


In [ ]:
import openai
from langchain.document_loaders import AzureBlobStorageContainerLoader
import numpy as np
import os

In [ ]:
openai.api_key = ""
openai.api_base = "https://testnew.openai.azure.com/" # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2022-12-01' # this may change in the future

deployment_name = 'ishita-davinci-test-3'

In [ ]:
print('Sending a test completion job')
start_phrase = 'Write a tagline for an ice cream shop. '
response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=10)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(start_phrase+text)

Sending a test completion job
Write a tagline for an ice cream shop. "The Coolest Treat in Town!"


In [ ]:
loaders = AzureBlobStorageContainerLoader(conn_str="DefaultEndpointsProtocol=https;AccountName=openaiblob1",container="statusreport")
documents = loaders.load()
print(f'You have {len(documents)} documents in your data')

text_to_summarize2=''
for doc in documents:
  text_to_summarize2 += doc.page_content
text_to_summarize2

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


You have 4 documents in your data


'    Status Timefra me:\n    10-Jul-2023 to 16-Jul-2023\n    Status Submitted on:\n    11-Jul-2023\n    Submitted Ev\n    Sareun Maur\n\nAll dates are as per the preferred tuliezone (TJTC-05-100)\n\nProject name\n\nSRN\': BofA GBAM - Quartz Migration_C .6908326161\n\nExecutive Status Summary\nStatus AMBER as on 10th July 2023.\n\nTotal Quartz release progress is at 89%\n\nMilestones re-baselined to Mid Aug - dueto platform carp incidents (Azcopy, DNS etc) - Overall status continuesto be Amber till the positive validation of defect remediations and workarounds; ETA 5/30\n\nBegan scale testing for data free pilot on a 10k node grid\n\nExecutive Summary:\n\nThe key priority of this project is to provide Bank of America v.17h a delivery team focused on the build out of the Azure infrastructure components required :c hos: Quartz and to help scale the Azure infrastructure to meet Quartz\'s scale requirements.\n\nInitial scope areas are:\n\nAssist with finalizing the overall Architecture of Q

In [ ]:
import math

words = text_to_summarize2.split(" ")
print(len(words))
split_len = len(words)//2500 + 1
chunks = np.array_split(words, split_len)
print(len(chunks), len(chunks[0]))

2944
2 1472


In [ ]:
count=0
def recursive_summarize(text_to_summarize, target_word_count=500):
    words = text_to_summarize.split(" ")
    split_len = len(words)//2500 + 1
    chunks = np.array_split(words, split_len)
    #chunks = np.array_split(words, 1)
    summary_responses = []
    full_summary = ""

    for chunk in chunks:
        sentences = ' '.join(list(chunk))
        prompt = f"Below are three weekly status reports for Project name. Extract the key project completion trends, key changes between each, Project Financial Summary: #Start of Report{sentences}#End of Report"
        response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.3,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=1
        )

        response_text = response["choices"][0]["text"]
        summary_responses.append(response_text+'\n\n')

    full_summary = "".join(summary_responses)
    word_count = len(full_summary.split())
    print(word_count)
    if word_count > target_word_count:
    #     print(++count)
        return recursive_summarize(full_summary, target_word_count)

    return full_summary

#Summary of each doc
#no_of_docs = len(text_to_summarize2)
#final_summary = []
#for i in range(no_of_docs):
  # initial_text_to_summarize = text_to_summarize2
  # final_summary.append(recursive_summarize(initial_text_to_summarize, target_word_count=50))

# Example usage
initial_text_to_summarize = text_to_summarize2
final_summary = recursive_summarize(initial_text_to_summarize, target_word_count=500)

160


### YE CORRECT HAI

In [ ]:
from docx import Document
import os
from azure.storage.blob import BlobServiceClient

# Azure Blob Storage credentials
conn_str = "DefaultEndpointsProtocol=https;AccountName=openaiblob1;"
container_name = "statusreport"

# Local directory to download documents
download_dir = "/tmp/downloaded_docs"

# Create the download directory if it doesn't exist
os.makedirs(download_dir, exist_ok=True)

# Create BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Get the container client
container_client = blob_service_client.get_container_client(container_name)

# Download the documents to the local directory
documents = []
for blob in container_client.list_blobs():
    file_path = os.path.join(download_dir, blob.name)
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, "wb") as file:
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob.name)
        data = blob_client.download_blob()
        file.write(data.readall())
    documents.append(file_path)

print(f"You have {len(documents)} documents in your data")

text_to_summarize2 = ""

for doc_path in documents:
    file_name = os.path.basename(doc_path)
    print(file_name)
    # Extract text and tabular data from .docx file
    if file_name.endswith(".docx"):
        document = Document(doc_path)
        for paragraph in document.paragraphs:
            text_to_summarize2 += paragraph.text + "\n"
        for table in document.tables:
            for row in table.rows:
                for cell in row.cells:
                    text_to_summarize2 += cell.text + "\n"

# Clean up the downloaded documents
for doc_path in documents:
    os.remove(doc_path)

print(text_to_summarize2)


You have 3 documents in your data
SRV_ BofA GBAM - Quartz Migration_C.6908326161_11-Jul-2023 (1).docx
SRV_ BofA GBAM - Quartz Migration_C.6908326161_11-Jul-2023 (2).docx
SRV_ BofA GBAM - Quartz Migration_C.6908326161_11-Jul-2023 (3).docx


All dates are as per the preferred tuliezone (TJTC-05-100)
Project name
SRN': BofA GBAM - Quartz Migration_C .6908326161

Total Quartz release progress is at 89%
Milestones re-baselined to Mid Aug - dueto platform carp incidents (Azcopy, DNS etc) - Overall status continuesto be Amber till the positive validation of defect remediations and workarounds; ETA 5/30
Began scale testing for data free pilot on a 10k node grid
Executive Summary:
The key priority of this project is to provide Bank of America v.17h a delivery team focused on the build out of the Azure infrastructure components required :c hos: Quartz and to help scale the Azure infrastructure to meet Quartz's scale requirements.
Initial scope areas are:
Assist with finalizing the overall Archit

In [ ]:
file_names = [os.path.basename(doc.metadata['source']) for doc in documents]
print(file_names)

['cricket summary.docx', 'football summary.docx', 'hockey summary.docx', 'tennis summary.docx']


In [ ]:
import math

words = text_to_summarize2.split(" ")
print(len(words))
split_len = len(words)//2500 + 1
chunks = np.array_split(words, split_len)
print(len(chunks), len(chunks[0]))

1954
1 1954


In [ ]:
text_to_summarize2

'\n\nAll dates are as per the preferred tuliezone (TJTC-05-100)\nProject name\nSRN\': BofA GBAM - Quartz Migration_C .6908326161\n\nTotal Quartz release progress is at 89%\nMilestones re-baselined to Mid Aug - dueto platform carp incidents (Azcopy, DNS etc) - Overall status continuesto be Amber till the positive validation of defect remediations and workarounds; ETA 5/30\nBegan scale testing for data free pilot on a 10k node grid\nExecutive Summary:\nThe key priority of this project is to provide Bank of America v.17h a delivery team focused on the build out of the Azure infrastructure components required :c hos: Quartz and to help scale the Azure infrastructure to meet Quartz\'s scale requirements.\nInitial scope areas are:\nAssist with finalizing the overall Architecture of Quartz Framework in Azure and identify all the subsystems necessary to build out a complete environment.\nConduct a series of Proof of Concepts (PoCs)to build, evaluate and finalize the design options for each sub

In [ ]:
count=0
def recursive_summarize(text_to_summarize, target_word_count=500):
    words = text_to_summarize.split(" ")
    split_len = len(words)//2500 + 1
    chunks = np.array_split(words, split_len)
    #chunks = np.array_split(words, 1)
    summary_responses = []
    full_summary = ""

    for chunk in chunks:
        sentences = ' '.join(list(chunk))
        prompt = f"Below are three weekly status reports for Project name. Extract the key project completion trends, key changes between each, Project Financial Summary: #Start of Report{sentences}#End of Report"
        response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.3,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=1
        )

        response_text = response["choices"][0]["text"]
        summary_responses.append(response_text+'\n\n')

    full_summary = "".join(summary_responses)
    word_count = len(full_summary.split())
    print(word_count)
    if word_count > target_word_count:
    #     print(++count)
        return recursive_summarize(full_summary, target_word_count)

    return full_summary

#Summary of each doc
#no_of_docs = len(text_to_summarize2)
#final_summary = []
#for i in range(no_of_docs):
  # initial_text_to_summarize = text_to_summarize2
  # final_summary.append(recursive_summarize(initial_text_to_summarize, target_word_count=50))

# Example usage
initial_text_to_summarize = text_to_summarize2
final_summary = recursive_summarize(initial_text_to_summarize, target_word_count=500)


81


In [ ]:
final_summary #Summary of full

'\n\nThe key project completion trends are that the total Quartz release progress is increasing from 89% to 91% to 94%, and the estimated time of completion is still Mid August. The key changes between each report are that the labor revenue, expense revenue, and hours have all increased slightly, and the AZ Copy failure issue has been resolved. The Project Financial Summary for each report shows that the contract amounts, actual amounts, and estimated to complete amounts have all increased slightly.\n\n'

In [ ]:
len(final_summary[0])

1